<h2>Regression Models with Keras</h2>
    
Last edited 10/27/2021 by Luis E. Valdivia.

<h3>Part A</h3> 


Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [2]:
import pandas as pd
import numpy as np

Let's download the data and read it into a <em>pandas</em> dataframe.


In [3]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Recall that the data is already clean and ready to use. Let's split data into predictors and target.

In [4]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Lastly, we normalize the data by substracting the mean and dividing by the standard deviation.

In [5]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to _n_cols_ since we will need this number when building our network.

In [6]:
n_cols = predictors_norm.shape[1] # number of predictors

Now we import Keras.

In [7]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [8]:
from keras.models import Sequential
from keras.layers import Dense

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [9]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has one hidden layers of 10 hidden units.

Let's call the function now to create our model.

In [10]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the fit method. We will leave out 30% of the data for validation and we will train the model for 50 epochs.

In [11]:
# fit the model
results = model.fit(predictors, target, validation_split=0.3, epochs=50, verbose=2)



Train on 721 samples, validate on 309 samples
Epoch 1/50


2021-10-27 23:39:57.595347: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2021-10-27 23:39:57.603244: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2593900000 Hz
2021-10-27 23:39:57.611710: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ab012afe00 executing computations on platform Host. Devices:
2021-10-27 23:39:57.611769: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2021-10-27 23:39:57.693705: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

 - 0s - loss: 21197.7712 - val_loss: 13824.0857
Epoch 2/50
 - 0s - loss: 12210.4318 - val_loss: 7093.3743
Epoch 3/50
 - 0s - loss: 6120.9966 - val_loss: 2662.7380
Epoch 4/50
 - 0s - loss: 2914.5830 - val_loss: 1268.8450
Epoch 5/50
 - 0s - loss: 2101.0688 - val_loss: 1105.7938
Epoch 6/50
 - 0s - loss: 1780.4362 - val_loss: 954.3121
Epoch 7/50
 - 0s - loss: 1535.8064 - val_loss: 846.0669
Epoch 8/50
 - 0s - loss: 1317.1399 - val_loss: 743.2846
Epoch 9/50
 - 0s - loss: 1095.0711 - val_loss: 631.0525
Epoch 10/50
 - 0s - loss: 891.2807 - val_loss: 533.6863
Epoch 11/50
 - 0s - loss: 715.3418 - val_loss: 480.5642
Epoch 12/50
 - 0s - loss: 568.2641 - val_loss: 457.6747
Epoch 13/50
 - 0s - loss: 487.0279 - val_loss: 442.7003
Epoch 14/50
 - 0s - loss: 451.0136 - val_loss: 432.8350
Epoch 15/50
 - 0s - loss: 433.0126 - val_loss: 418.4749
Epoch 16/50
 - 0s - loss: 417.3392 - val_loss: 412.8043
Epoch 17/50
 - 0s - loss: 408.0978 - val_loss: 406.3547
Epoch 18/50
 - 0s - loss: 400.7905 - val_loss: 396.

We want to do this 50 times and record the mean square error, so we will make an array of size 50 to keep track of the errors and train/test the model with verbose set to 0.

In [12]:
mean_errors = np.zeros(50)

for x in range(50):
    model = regression_model()
    results = model.fit(predictors, target, validation_split=0.3, epochs=50, verbose=0)
    mean_errors[x] = results.history['val_loss'][-1]
    print("Run Number:", x)

Run Number: 0
Run Number: 1
Run Number: 2
Run Number: 3
Run Number: 4
Run Number: 5
Run Number: 6
Run Number: 7
Run Number: 8
Run Number: 9
Run Number: 10
Run Number: 11
Run Number: 12
Run Number: 13
Run Number: 14
Run Number: 15
Run Number: 16
Run Number: 17
Run Number: 18
Run Number: 19
Run Number: 20
Run Number: 21
Run Number: 22
Run Number: 23
Run Number: 24
Run Number: 25
Run Number: 26
Run Number: 27
Run Number: 28
Run Number: 29
Run Number: 30
Run Number: 31
Run Number: 32
Run Number: 33
Run Number: 34
Run Number: 35
Run Number: 36
Run Number: 37
Run Number: 38
Run Number: 39
Run Number: 40
Run Number: 41
Run Number: 42
Run Number: 43
Run Number: 44
Run Number: 45
Run Number: 46
Run Number: 47
Run Number: 48
Run Number: 49


In [15]:
print("The mean of the mean squared errors is:", 
      mean_errors.mean())
print("The standard deviation of the mean squared errors is:", 
      mean_errors.std())

The mean of the mean squared errors is: 506.68839601411787
The standard deviation of the mean squared errors is: 484.4251867469845


<h3>Part B</h3> 

Now we repeat Part A but with the normalized data instead.

In [16]:
mean_errors_normed = np.zeros(50)

for x in range(50):
    model = regression_model()
    results = model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=0)
    mean_errors_normed[x] = results.history['val_loss'][-1]
    print("Run Number:", x)

Run Number: 0
Run Number: 1
Run Number: 2
Run Number: 3
Run Number: 4
Run Number: 5
Run Number: 6
Run Number: 7
Run Number: 8
Run Number: 9
Run Number: 10
Run Number: 11
Run Number: 12
Run Number: 13
Run Number: 14
Run Number: 15
Run Number: 16
Run Number: 17
Run Number: 18
Run Number: 19
Run Number: 20
Run Number: 21
Run Number: 22
Run Number: 23
Run Number: 24
Run Number: 25
Run Number: 26
Run Number: 27
Run Number: 28
Run Number: 29
Run Number: 30
Run Number: 31
Run Number: 32
Run Number: 33
Run Number: 34
Run Number: 35
Run Number: 36
Run Number: 37
Run Number: 38
Run Number: 39
Run Number: 40
Run Number: 41
Run Number: 42
Run Number: 43
Run Number: 44
Run Number: 45
Run Number: 46
Run Number: 47
Run Number: 48
Run Number: 49


In [20]:
print("The mean of the mean squared errors, with normalization, is:",
      mean_errors_normed.mean())
print("The standard deviation of the mean squared errors, with normalization, is:", 
      mean_errors_normed.std())
print("Clearly both have decreased compared to Part A.")

The mean of the mean squared errors, with normalization, is: 369.3625798256884
The standard deviation of the mean squared errors, with normalization, is: 92.12537895670323
Clearly both have decreased compared to Part A.


<h3>Part C</h3> 

Now we repeat Part B but with 100 epochs instead.

In [18]:
mean_errors_normed_100 = np.zeros(50)

for x in range(50):
    model = regression_model()
    results = model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=0)
    mean_errors_normed_100[x] = results.history['val_loss'][-1]
    print("Run Number:", x)

Run Number: 0
Run Number: 1
Run Number: 2
Run Number: 3
Run Number: 4
Run Number: 5
Run Number: 6
Run Number: 7
Run Number: 8
Run Number: 9
Run Number: 10
Run Number: 11
Run Number: 12
Run Number: 13
Run Number: 14
Run Number: 15
Run Number: 16
Run Number: 17
Run Number: 18
Run Number: 19
Run Number: 20
Run Number: 21
Run Number: 22
Run Number: 23
Run Number: 24
Run Number: 25
Run Number: 26
Run Number: 27
Run Number: 28
Run Number: 29
Run Number: 30
Run Number: 31
Run Number: 32
Run Number: 33
Run Number: 34
Run Number: 35
Run Number: 36
Run Number: 37
Run Number: 38
Run Number: 39
Run Number: 40
Run Number: 41
Run Number: 42
Run Number: 43
Run Number: 44
Run Number: 45
Run Number: 46
Run Number: 47
Run Number: 48
Run Number: 49


In [21]:
print("The mean of the mean squared errors, with normalization and 100 epochs, is:", 
      mean_errors_normed_100.mean())
print("The standard deviation of the mean squared errors, with normalization and 100 epochs, is:", 
      mean_errors_normed_100.std())
print("Clearly both have decreased compared to Part B.")

The mean of the mean squared errors, with normalization and 100 epochs, is: 185.0299294876049
The standard deviation of the mean squared errors, with normalization and 100 epochs, is: 38.95524106971021
Clearly both have decreased compared to Part B.


<h3>Part D</h3> 

Now we rebuild our regression model to have 3 hidden layers, each of 10 nodes and ReLU activation function.

In [22]:
# define regression model
def regression_model_2():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Now we repeat Part B but with this deeper model instead.

In [23]:
mean_errors_deeper = np.zeros(50)

for x in range(50):
    model = regression_model_2()
    results = model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=0)
    mean_errors_deeper[x] = results.history['val_loss'][-1]
    print("Run Number:", x)

Run Number: 0
Run Number: 1
Run Number: 2
Run Number: 3
Run Number: 4
Run Number: 5
Run Number: 6
Run Number: 7
Run Number: 8
Run Number: 9
Run Number: 10
Run Number: 11
Run Number: 12
Run Number: 13
Run Number: 14
Run Number: 15
Run Number: 16
Run Number: 17
Run Number: 18
Run Number: 19
Run Number: 20
Run Number: 21
Run Number: 22
Run Number: 23
Run Number: 24
Run Number: 25
Run Number: 26
Run Number: 27
Run Number: 28
Run Number: 29
Run Number: 30
Run Number: 31
Run Number: 32
Run Number: 33
Run Number: 34
Run Number: 35
Run Number: 36
Run Number: 37
Run Number: 38
Run Number: 39
Run Number: 40
Run Number: 41
Run Number: 42
Run Number: 43
Run Number: 44
Run Number: 45
Run Number: 46
Run Number: 47
Run Number: 48
Run Number: 49


In [24]:
print("The mean of the mean squared errors, with 3 hidden layers, is:", 
      mean_errors_deeper.mean())
print("The standard deviation of the mean squared errors, with 3 hidden layers, is:", 
      mean_errors_deeper.std())
print("Clearly both have decreased compared to Part B.")

The mean of the mean squared errors, with 3 hidden layers, is: 143.30192322271157
The standard deviation of the mean squared errors, with 3 hidden layers, is: 20.089202139581598
Clearly both have decreased compared to Part B.
